# Analysis of Bias and Variance in a linear regression model
In this notebook, using a salary prediction dataset called salary prediction dataset https://www.kaggle.com/datasets/rkiattisak/salaly-prediction-for-beginer 
,we will analyze the bias and prediction using different features of the dataset

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder

In [ ]:
# Read the data

salary_data = pd.read_csv("data/Salary Data.csv")

In [ ]:
salary_data.head()

In [ ]:
# Check for missing values
salary_data.isnull().sum().sum()

In [ ]:
# Drop any rows that have missing values
salary_data = salary_data.dropna()
salary_data.isnull().sum().sum()

### We want to build a model that can predict salary based on the following features:
- Age
- Educational Level
- Years of Experience
- Gender
- Job Title

In [ ]:
features = ['Age', 'Gender', 'Education Level', 'Years of Experience', 'Job Title', 'Salary']
salary_data = salary_data[features]
salary_data.head()

### Let's examine the categorical columns

In [ ]:
print(salary_data['Gender'].unique())
print(salary_data['Education Level'].unique())
print(salary_data['Job Title'])

### We will use LabelEncoder to conver Categorical values to numeric

In [ ]:
def Encoder (df):
    columnsToEncode = list(df.select_dtypes(include = ['category', 'object']))
    le = LabelEncoder()
    for feature in columnsToEncode:
        try:
            df[feature] = le.fit_transform(df[feature])
        except:
            print('Error encoding '+feature)
    return df

In [ ]:
categorical_features = ['Gender', 'Education Level', 'Job Title']
salary_data = Encoder(salary_data)
salary_data.head()

## Regression Model
### 1. Let's try with only one feature (input), for example years of experience 

In [ ]:
df_binary = salary_data[['Years of Experience', 'Salary']]

### 1.1 Explore the data

In [ ]:
sns.lmplot(x = 'Years of Experience', y='Salary', data = df_binary, order = 2, ci = None)
plt.show()

### 1.2 Train our model

In [ ]:
# Separate data into indpendent (X) and dependent variables 
# Convert the data frame into a numpy array since each dataframe contains only one column
X = np.array(df_binary['Years of Experience']).reshape(-1,1)
y = np.array(df_binary['Salary']).reshape(-1,1) 

In [ ]:
# Define a function to split the data
def prepare_train_cv_test (X, y):
    
    # get 60% of the dataset as the training set.  Put the remaining 40% in temporary variables
    X_train, X_, y_train, y_ = train_test_split(X, y, test_size = 0.40, random_state = 55)

    # Split the 40% subset above into two: one half for cross validation and the other for the test set
    X_cv, X_test, y_cv, y_test = train_test_split(X_, y_,  test_size = .50, random_state = 55)
    
    return X_train, y_train, X_cv, y_cv, X_test, y_test

In [ ]:
X_train, y_train, X_cv, y_cv, X_test, y_test = prepare_train_cv_test(X, y) 

In [ ]:
model= LinearRegression()
model.fit(X_train, y_train)
y_pred_cv = model.predict(X_cv)
y_pred_train = model.predict(X_train)


# The coefficients
print("Coefficients: \n", model.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_cv, y_pred_cv))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_cv, y_pred_cv))

# Plot outputs
plt.scatter(X_cv, y_cv, color="black")
plt.plot(X_cv, y_pred_cv, color="blue", linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
train_mse = mean_squared_error(y_train, y_pred_train)
cv_mse = mean_squared_error(y_cv, y_pred_cv) 
print(f"Training MSE : {train_mse:.0f}")
print(f"Cross Validation MSE: {cv_mse: .0f}")

#### The results show that the both training and Cross Validation Errors are high. This maybe due to high bias (underfit). We could one or more of the following:

- Try adding additional features
- Try decreasing the regulatiztion parameter
- Try adding polynomial features

We will try the first option


### 2.1 Let's try adding more features, age, gender

In [ ]:
X = salary_data[['Age', 'Years of Experience', 'Gender', 'Job Title']]
y = salary_data['Salary']

### We will now split the data into train, cross validation, and test sets

In [ ]:
# Define a function to split the data
def prepare_train_cv_test (X, y):
    
    # get 60% of the dataset as the training set.  Put the remaining 40% in temporary variables
    X_train, X_, y_train, y_ = train_test_split(X, y, test_size = 0.40, random_state = 55)

    # Split the 40% subset above into two: one half for cross validation and the other for the test set
    X_cv, X_test, y_cv, y_test = train_test_split(X_, y_,  test_size = .50, random_state = 55)
    
    return X_train, y_train, X_cv, y_cv, X_test, y_test

In [ ]:
X_train, y_train, X_cv, y_cv, X_test, y_test = prepare_train_cv_test (X, y)

In [ ]:
print(f" Training Data shape: {X_train.shape}")
print(f" Cross Validation Data shape: {X_cv.shape}")
print(f" Test Data shape: {X_test.shape}")

In [ ]:
# The data has different range of values, for example age and years of experience have a large range than 
# Gender and Educational Level.  We want to scale the data into new values that are easier to compare
scale = StandardScaler()
X_train_scaled = scale.fit_transform(X_train)
X_cv_scaled = scale.fit_transform(X_cv)
X_test_scaled = scale.fit_transform(X_test)

###  2.2 Train our model

In [ ]:
model.fit(X_train_scaled, y_train)

### 2.3 Explore the results

In [ ]:
y_pred_cv = model.predict(X_cv_scaled)
y_pred_train = model.predict(X_train_scaled)

# The coefficients
print("Coefficients: \n", model.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_cv, y_pred_cv))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_cv, y_pred_cv))

In [ ]:

y_pred_train = model.predict(X_train_scaled)
train_mse = mean_squared_error(y_train, y_pred_train)
cv_mse = mean_squared_error(y_cv, y_pred_cv)
print(f"Training MSE : {train_mse:.0f}")
print(f"Cross Validation MSE: {cv_mse: .0f}")

The accuracy the same

The model is performing better.  Both training and Cross Validation MSE were decreased.  Can we improve it furthere?

### 3.1 Let's try addition polynomial features

In [ ]:
poly = PolynomialFeatures(degree=3, include_bias=False)
X_train_mapped = poly.fit_transform(X_train)
X_cv_mapped = poly.fit_transform(X_cv)
X_test_mapped = poly.fit_transform(X_test)

In [ ]:
X_train_mapped_scaled = scale.fit_transform(X_train_mapped)
X_cv_mapped_scaled = scale.fit_transform(X_cv_mapped)
X_test_mapped_scaled = scale.fit_transform(X_test_mapped)

###  3.2 Train our model

In [ ]:
model.fit(X_train_mapped_scaled, y_train)


### 3.3 Explore the results

In [ ]:
y_pred_cv = model.predict(X_cv_mapped_scaled)
y_pred_train = model.predict(X_train_mapped_scaled)

# The coefficients
print("Coefficients: \n", model.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_cv, y_pred_cv))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_cv, y_pred_cv))

In [ ]:
y_pred_cv = model.predict(X_cv_mapped_scaled)
y_pred_train = model.predict(X_train_mapped_scaled)
train_mse = mean_squared_error(y_train, y_pred_train)
cv_mse = mean_squared_error(y_cv, y_pred_cv)
print(f"Training MSE : {train_mse:.0f}")
print(f"Cross Validation MSE: {cv_mse: .0f}")

The model is performing even better.  Both training and Cross Validation MSE were decreased.

### 4.1 Let's try various degrees of polynomial to see if our model gets more accurate

In [ ]:
def train_plot_poly(model, x_train, y_train, x_cv, y_cv, max_degree=10, baseline=None):
    
    train_mses = []
    cv_mses = []
    models = []
    scalers = []
    degrees = range(1,max_degree+1)

    # Loop over 10 times. Each adding one more degree of polynomial higher than the last.
    for degree in degrees:
        print(f"Dgree {degree}:")
        # Add polynomial features to the training set
        poly = PolynomialFeatures(degree, include_bias=False)
        X_train_mapped = poly.fit_transform(x_train)

        # Scale the training set
        scaler_poly = StandardScaler()
        X_train_mapped_scaled = scaler_poly.fit_transform(X_train_mapped)
        scalers.append(scaler_poly)

        # Create and train the model
        model.fit(X_train_mapped_scaled, y_train )
        models.append(model)

        # Compute the training MSE
        yhat = model.predict(X_train_mapped_scaled)
        train_mse = mean_squared_error(y_train, yhat)
        print(f"     Train mse: {train_mse: .0f}")
        train_mses.append(train_mse)

        # Add polynomial features and scale the cross-validation set
        poly = PolynomialFeatures(degree, include_bias=False)
        X_cv_mapped = poly.fit_transform(x_cv)
        X_cv_mapped_scaled = scaler_poly.transform(X_cv_mapped)

        # Compute the cross-validation MSE
        yhat = model.predict(X_cv_mapped_scaled)
        cv_mse = mean_squared_error(y_cv, yhat)
        print(f"     cv mse: {cv_mse: .0f}")
        cv_mses.append(cv_mse)
        print("      Coefficient of determination (r2 score): %.2f" % r2_score(y_cv, yhat))

    # Plot the results
    
    plt.plot(degrees, train_mses, marker='o', c='r', label='training MSEs'); 
    plt.plot(degrees, cv_mses, marker='o', c='b', label='CV MSEs'); 
    plt.plot(degrees, np.repeat(baseline, len(degrees)), linestyle='--', label='baseline')
    plt.title("degree of polynomial vs. train and CV MSEs")
    plt.xticks(degrees)
    plt.xlabel("degree"); 
    plt.ylabel("MSE"); 
    plt.legend()
    plt.show()

In [ ]:
X_train, y_train, X_cv, y_cv, X_test, y_test = prepare_train_cv_test (X, y)

In [ ]:
print(f" Training Data shape: {X_train.shape}")
print(f" Cross Validation Data shape: {X_cv.shape}")
print(f" Test Data shape: {X_test.shape}")

In [ ]:
train_plot_poly(model, X_train, y_train, X_cv, y_cv, max_degree =10, baseline= 122122122)

Based on the above results polynomial of degree 3 seems to be best fit